# ETL Pipeline - Generated Code
        
**Generated:** 2025-09-04 17:37:10  
**Configuration:** Text: Text file (5746 chars)

## Overview
This notebook contains the auto-generated ETL pipeline code for migrating data from Oracle to Databricks Delta Lake.


In [ ]:
import os
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, isnull

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Load environment variables
DB_HOST = os.environ['DB_HOST']
DB_PORT = os.environ['DB_PORT']
DB_USER = os.environ['DB_USER']
DB_PASSWORD = os.environ['DB_PASSWORD']
DB_NAME = os.environ['DB_NAME']

# Create SparkSession with Delta Lake extensions
def create_spark_session():
    spark = SparkSession.builder \
        .appName("Sales ETL Pipeline") \
        .config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
        .getOrCreate()
    return spark

# Load data from database
def load_data(spark):
    try:
        # Load customers table
        customers_df = spark.read.format("jdbc") \
            .option("url", f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", "SALES_DB.CUSTOMERS") \
            .option("user", DB_USER) \
            .option("password", DB_PASSWORD) \
            .load() \
            .filter(col("STATUS") == "ACTIVE")

        # Load products table
        products_df = spark.read.format("jdbc") \
            .option("url", f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", "SALES_DB.PRODUCTS") \
            .option("user", DB_USER) \
            .option("password", DB_PASSWORD) \
            .load()

        # Load sales table
        sales_df = spark.read.format("jdbc") \
            .option("url", f"jdbc:postgresql://{DB_HOST}:{DB_PORT}/{DB_NAME}") \
            .option("driver", "org.postgresql.Driver") \
            .option("dbtable", "SALES_DB.SALES") \
            .option("user", DB_USER) \
            .option("password", DB_PASSWORD) \
            .load() \
            .filter((col("QUANTITY") > 0) & (col("TOTAL_AMOUNT") > 0))

        logger.info("Data loaded successfully")
        return customers_df, products_df, sales_df

    except Exception as e:
        logger.error(f"Error loading data: {e}")
        raise

# Perform ETL operations
def etl_data(customers_df, products_df, sales_df):
    try:
        # Join sales with customers and products using broadcast
        sales_df = sales_df.join(broadcast(customers_df), "CUSTOMER_ID", "inner")
        sales_df = sales_df.join(broadcast(products_df), "PRODUCT_ID", "inner")

        # Perform monthly aggregation
        aggregated_df = sales_df.groupBy(col("SALE_DATE").substr(1, 7).alias("MONTH"), "CUSTOMER_ID", "PRODUCT_ID") \
            .agg(count("SALE_ID").alias("SALES_COUNT"), sum("TOTAL_AMOUNT").alias("TOTAL_SALES"))

        logger.info("ETL operations completed successfully")
        return aggregated_df

    except Exception as e:
        logger.error(f"Error performing ETL operations: {e}")
        raise

# Write data to Delta Lake
def write_data(aggregated_df):
    try:
        # Write data to Delta Lake with partitioning
        aggregated_df.write.format("delta") \
            .partitionBy("MONTH") \
            .mode("overwrite") \
            .save("delta://sales_data")

        logger.info("Data written to Delta Lake successfully")
    except Exception as e:
        logger.error(f"Error writing data to Delta Lake: {e}")
        raise

# Perform data quality checks
def data_quality_checks(aggregated_df):
    try:
        # Check for null values
        null_counts = aggregated_df.select([count(isnull(c)).alias(c) for c in aggregated_df.columns]).collect()[0]
        logger.info(f"Null counts: {null_counts}")

        # Check row count
        row_count = aggregated_df.count()
        logger.info(f"Row count: {row_count}")

    except Exception as e:
        logger.error(f"Error performing data quality checks: {e}")
        raise

# Main function
def main():
    spark = create_spark_session()
    customers_df, products_df, sales_df = load_data(spark)
    aggregated_df = etl_data(customers_df, products_df, sales_df)
    write_data(aggregated_df)
    data_quality_checks(aggregated_df)

if __name__ == "__main__":
    main()

## Validation Report

**Summary:** 9/9 checks passed

| Check | Status | Details |
|-------|--------|---------|
| SparkSession | PASS PASS | SparkSession properly initialized |
| Delta Lake | PASS PASS | Delta Lake format detected |
| Environment Variables | PASS PASS | Uses environment variables |
| No Hardcoded Creds | PASS PASS | No hardcoded credentials found |
| Predicate Pushdown | PASS PASS | Database-level filtering detected |
| Broadcast Joins | PASS PASS | Broadcast joins implemented |
| Error Handling | PASS PASS | Exception handling present |
| Logging | PASS PASS | Logging implemented |
| Data Quality Checks | PASS PASS | Data quality checks present |


## Test Report

**Summary:** 5/6 tests passed

| Test | Status | Input | Expected | Output |
|------|--------|-------|----------|--------|
| Syntax Validation | PASS | Python code compilation | Valid Python syntax | Code compiles successfully |
| Business Rules Filter | PASS | 3 records with mixed status/values | 1 valid record | 1 records after filtering |
| Data Transformation | PASS | Sales with dates | Year/month extraction | 2 unique year-month combinations |
| Aggregation Logic | PASS | 4 records to aggregate | Customer 1, Product 10: qty=8, amt=80 | Aggregation produces 3 groups |
| Data Volume Handling | PASS | Simulated 1,000,000 records | Handles large volumes | Volume test passed |
| Performance Optimizations | FAIL | Code analysis | Performance features | Found: broadcast joins |
